# Setup NGINX with HTTPS

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes also your Vault server is already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables

This also assumes you have the Vault CLI installed.

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-v $LOGS_PATH:/vault_logs \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} \
-p 5696:5696 hashicorp/vault-enterprise:latest

In [ ]:
# Install nginx for testing on MacOS.  Assumes you have brew installed.
# If already installed, skip this step.
brew install nginx

In [ ]:
# Copy the nginx.conf that is for SSL
yes | cp nginx.conf /opt/homebrew/etc/nginx/nginx.conf

## Setup Vault as a CA and Issue TLS certificate to NGINX

In [ ]:
# Enable PKI engine at the default mount path
vault secrets enable pki
# Enable approle Auth
vault auth enable approle

In [ ]:
# configure max lease ttl on certificates - 8760h or 365 days
vault secrets tune -max-lease-ttl=8760h pki

In [ ]:
# Create the CA, the private key is kept inside Vault.
# Alternatively, generate an intermediate CA and sign with your root CA.
# CA cert is valid for 8760h or 365 days
vault write pki/root/generate/internal \
    common_name=my-org.com \
    ttl=8760h

In [ ]:
# configure Vault with the URL's for CRL
vault write pki/config/urls \
    issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
    crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"

## Import the CA cert to your local machine

In [ ]:
# Store the CA certificate to import into your keychain.  Using keychain app to import in.
curl http://127.0.0.1:8200/v1/pki/ca_chain > ca.crt
open ca.crt


In [ ]:
# Note: To avoid cert warnings, open keychain app and configure CA cert for my-org.com to Always Trust in the properties
open -a "keychain Access.app"

<img src="images/ca-trust-keychain-settings.png">

In [ ]:
# configure web certificate for localhost.  Set TTL to 30s for demo.
vault write pki/roles/website1role \
    allowed_domains=localhost \
    allow_subdomains=true \
    max_ttl=30s

In [ ]:
# Generate the web cert and place it in the nginx folder
vault write -format=json pki/issue/website1role \
    common_name=localhost > cert.json
jq -r .data.certificate < cert.json > /opt/homebrew/etc/nginx/cert.pem
jq -r .data.private_key < cert.json > /opt/homebrew/etc/nginx/cert.key




In [ ]:
# By default, Homebrew installs NGINX in /usr/local/Cellar/nginx/<version> 
# and creates symbolic links at /usr/local/opt/. The nginx binary has a 
# symbolic link in /usr/local/bin and this folder is already included in the $PATH.
# configuration file will be in /opt/homebrew/etc/nginx/nginx.conf
# sudo systemctl start nginx # linux
nginx -s stop # MacOS
nginx # MacOS

## Verifying the Certificate Validity

In [ ]:
# open HTTPS://localhost # open using default browser
# Test nginx using Chrome in incognito mode otherwise the cert expiry doesn't seem to show due to some caching.
# Show that the website is loaded using SSL
# Close the browser and wait for 30 secs and open the page again to show that the cert date is now expired.
open -na "Google Chrome" --args -incognito HTTPS://localhost

# Leave the browser window open and continue with the Vault Agent setup.

# Setup Vault Agent to Refresh the NGINX PKI cert

In [ ]:
# Create the policy for the vault agent to generate certificates and to authenticate with tokens
vault policy write pki-agent-policy - << EOF
path "pki/issue/website1role" {
  capabilities = ["create", "update"]
}
path "auth/token/*" {
  capabilities = ["create", "update"]
}
EOF

In [ ]:
# Create the approle for the agent with the above policy
# Secret ID TTL is currently set to 10m validity.
vault write auth/approle/role/pki-agent-app-role \
    secret_id_ttl=10m \
    token_num_uses=0 \
    token_ttl=20m \
    token_max_ttl=30m \
    secret_id_num_uses=40 \
    token_policies=pki-agent-policy,default
# Show settings on app role
vault read auth/approle/role/pki-agent-app-role

In [ ]:
# Store roleid value for Vault Agent
vault read -field=role_id auth/approle/role/pki-agent-app-role/role-id > roleid

In [ ]:
# Store secretid value for Vault Agent
vault write -f -field=secret_id auth/approle/role/pki-agent-app-role/secret-id > secretid

In [ ]:
# Run the vault agent.  Show in the previous launch browser that the cert expiry date changes every 30 secs.  
# After demo, stop this cell and cleanup.
vault agent -config agent.hcl

In [ ]:
# After you stop the Vault Agent, you can show after the 30 sec expiry that the TLS cert is no longer refreshed 
# and the website hits an error.
open -na "Google Chrome" --args -incognito HTTPS://localhost

# Cleanup

In [ ]:
# Cleanup

# stop nginx
nginx -s stop # MacOS
#sudo systemctl stop nginx  # linux

rm cert.json
rm agent-token
rm roleid
rm secretid
rm ca.crt
vault secrets disable pki
vault auth disable approle

In [ ]:
# Manual Cleanup step
open -a "keychain Access.app"
# Remove the my-org.com certificate

In [ ]:
# Optional.  Other commands.
# reload nginx cert
#sudo systemctl reload nginx  # linux
#nginx -s reload # MacOS